Analysis of problem:

Goal:

Find an off ball speed prediction from the given variables
speed_A (more reliable than b (note "quirks"))
speed_B (less reliable) (find relationship between these 2)

---


vangle_A (launch angle from system A)
vangle_B (launch angle from system B)



In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression

In [ ]:
battedBalls = pd.read_csv('battedBallData.csv')

In [ ]:
battedBalls['combineVangles'] = battedBalls['vangle_A'].fillna(battedBalls['vangle_B'])
inconsistent = battedBalls[
    ((battedBalls['hittype']== 'ground_ball') & (battedBalls['combineVangles']>0)) |
    ((battedBalls['hittype'].isin(['fly_ball', 'popup']))&(battedBalls['combineVangles']<0))]
print(f"{len(inconsistent)}")
battedBallsClean = battedBalls.drop(inconsistent.index)
# Some hit angles contradicts with their description. This segment is for filtering out groundouts where the angle for system A or B are positive and fly balls, pop outs is negative
# Initially had this lower (a few blocks down), but wanted to get a better RMSE by cleaning beforehand

In [ ]:
model = LinearRegression()
combinedSpeeds = battedBallsClean.dropna(subset=['speed_A', 'speed_B'])
x = combinedSpeeds[['speed_B']]
y = combinedSpeeds['speed_A']
model.fit(x, y)
reg1 =model.intercept_
reg2 = model.coef_[0]
print(reg1, reg2)

speed_A = 40.10171042427472  + 0.60957283656058 * speed_B

In [ ]:
error = y - model.predict(x)
print(np.sqrt((error**2).mean())) # root mean square deviation
rmse = np.sqrt((error**2).mean()) # Close enough to 0 compared to the range for the sake of this task

In [ ]:
battedBallsClean['noSpeed_A']= reg1+reg2*battedBallsClean['speed_B'] # Changing the dataset for instances where there is no speed_A but there is a speed_B (Since we found the linear regression model for the speeds)
battedBallsClean['allAs'] = battedBallsClean['speed_A'].fillna(battedBallsClean['noSpeed_A']) # Filling missing values of speed_A
# battedBalls['combineVangles'] = battedBalls['vangle_A'].fillna(battedBalls['vangle_B']) # Moved higher to calculate RMSE off of "cleaned" rows

In [ ]:
Predmodel = battedBallsClean.dropna(subset=['allAs', 'combineVangles', 'hittype', 'batter'])

In [ ]:
mixedModel = smf.mixedlm('allAs~combineVangles+hittype', Predmodel,groups=Predmodel['batter'])
mixedModelEffects = mixedModel.fit()
effects = mixedModelEffects.random_effects
batterEffects = pd.DataFrame({
    'batter': effects.keys(),
    'batterEffect': [effect.values[0] for effect in effects.values()]}).reset_index(drop=True)
batterIntercept = mixedModelEffects.fe_params['Intercept']
batterEffects['Prediction'] = batterIntercept + batterEffects['batterEffect']

In [ ]:
batterAmt = Predmodel['batter'].value_counts().reset_index()
batterAmt.columns = ['batter', 'plays observed']
batterEffectsFrame = batterEffects.merge(batterAmt, on='batter') # Creating a dataframe to visualize the predictions

In [ ]:
randomEffectVariance = mixedModelEffects.cov_re.iloc[0, 0]
residualVariance = mixedModelEffects.scale
shrinkage = randomEffectVariance/(randomEffectVariance + (residualVariance/batterEffectsFrame['plays observed']))
shrinkageEffect = shrinkage * batterEffectsFrame['batterEffect']
batterEffectsFrame['Prediction'] = batterIntercept + shrinkageEffect # Weighted variance for a given player

In [ ]:
projections = batterEffectsFrame.sort_values(by='Prediction')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
projections